In [1]:
import pandas as pd
import numpy as np
import os
import glob
from geopy.geocoders import Nominatim
from IPython.display import display

In [2]:
# prevent data truncation
pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', None)

In [3]:
def read(dataset):
    """
    reads TSV files for all months from a dataset

    parameters:
        dataset (str): name of the dataset (sub, tag, or num)

    returns:
        DataFrame: a single DataFrame combining all TSV files for the dataset across months
    """

    files = []
    dfs = []
    
    for month in range(1, 13):
        files.append(f"financial statement and notes data sets (2024)/{month:02}/{dataset}.tsv")
    
    for file in files:
        df = pd.read_csv(file, sep='\t', low_memory=False)
        dfs.append(df)
    
    return pd.concat(dfs, axis=0)
    
# read the submissions dataset
sub = read('sub')
# read the tags dataset
tag = read('tag')
# read the numbers dataset
num = read('num')

In [ ]:
num = num.merge(sub[['adsh', 'name']], on='adsh', how='left')

In [5]:
# drop international data
sub = sub[sub['countryba'] == 'US']
num = num[num['adsh'].isin(sub['adsh'])]

# drop duplicate data
sub = sub.drop_duplicates(subset='name', keep='last')
tag = tag.drop_duplicates(subset='tag', keep='last')

In [6]:
# list of tags representing environmental-justice-related data
tags = ['GrossProfit', 'AccrualForEnvironmentalLossContingencies', 'EnvironmentalRemediationExpense', 'CharitableContributions', 'AmountCommittedForFundingSocialAndInfrastructureImprovementProjects']

sub[tags] = np.nan

In [7]:
def update_sub(tag, name):
    """
    updates the 'sub' DataFrame with values from the 'num' DataFrame based on 'name' and 'tag'

    parameters:
        tag (str): tag used to filter data in the 'num' DataFrame.
        name (str): registrant name used to filter data in both the 'num' and 'sub' DataFrames.

    returns:
        none: this function updates the 'sub' DataFrame in place.
    """
    
    value = values.get((name, tag), np.nan)
    print(f"{tag}: {value}")

    sub.loc[sub['name'] == name, tag] = value
    print(f"updated {tag} for {name}.")

# precompute tag values from 'num' DataFrame to reduce operations
values = {}
for (name, tag), group in num.groupby(['name', 'tag']):
    values[(name, tag)] = group['value'].sum()

# loop through each tag and unique name in 'sub' to update the 'sub' DataFrame
for tag in tags:
    for name in sub['name']:
        update_sub(tag, name)

GrossProfit: nan
updated GrossProfit for MOUNTAIN CREST ACQUISITION CORP. IV.
GrossProfit: nan
updated GrossProfit for RPT REALTY.
GrossProfit: nan
updated GrossProfit for ENDONOVO THERAPEUTICS, INC..
GrossProfit: nan
updated GrossProfit for SPARX HOLDINGS GROUP, INC..
GrossProfit: nan
updated GrossProfit for FIDELITY PRIVATE CREDIT CENTRAL FUND LLC.
GrossProfit: nan
updated GrossProfit for LIVENT CORP..
GrossProfit: nan
updated GrossProfit for SCHNITZER STEEL INDUSTRIES, INC..
GrossProfit: nan
updated GrossProfit for PRESIDENTIAL REALTY CORP/DE/.
GrossProfit: nan
updated GrossProfit for THIRDLINE REAL ESTATE INCOME FUND.
GrossProfit: nan
updated GrossProfit for FG FINANCIAL GROUP, INC..
GrossProfit: nan
updated GrossProfit for CHICO'S FAS, INC..
GrossProfit: nan
updated GrossProfit for DUKE ENERGY CAROLINAS, LLC.
GrossProfit: nan
updated GrossProfit for SOMALOGIC, INC..
GrossProfit: nan
updated GrossProfit for STARTEK, INC..
GrossProfit: nan
updated GrossProfit for NUVEEN AMT-FREE QUA

IOPub data rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_data_rate_limit`.

Current values:
ServerApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
ServerApp.rate_limit_window=3.0 (secs)



In [8]:
# drop unnecessary columns from 'sub' DataFrame
sub.drop(['countryma', 'stprma', 'cityma', 'zipma', 'mas1', 'mas2', 'countryinc', 'stprinc', 'ein', 'former',
         'changed', 'accepted', 'instance', 'nciks', 'aciks', 'floataxis', 'floatmems'], axis=1, inplace=True)

# rename remaining columns in 'sub' DataFrame
sub.columns = [
    'Accession Number', 
    'Central Index Key (CIK)', 
    'Name', 
    'Standard Industrial Classification (SIC)', 
    'Country', 
    'State', 
    'City', 
    'Zip Code', 
    'Address',
    'Address (Cont.)',
    'Phone Number', 
    'Filer Status', 
    'Well Known Seasoned Issuer', 
    'Fiscal Year End Date', 
    'Form', 
    'Balance Sheet Date', 
    'Fiscal Year Focus', 
    'Fiscal Period Focus', 
    'Date of Filing', 
    'Previous Report', 
    'Detail', 
    'Public Float', 
    'Date of Public Float', 
    'Gross Profit', 
    'Accrual for Environmental Loss Contingencies', 
    'Environmental Remediation Expenses', 
    'Charitable Contributions', 
    'Amount Committed for Funding Social and Infrastructure Improvement Projects'
]

# convert zeros to NaNs
sub['Gross Profit'] = sub['Gross Profit'].replace(0, np.nan)
sub['Accrual for Environmental Loss Contingencies'] = sub['Accrual for Environmental Loss Contingencies'].replace(0, np.nan)
sub['Environmental Remediation Expenses'] = sub['Environmental Remediation Expenses'].replace(0, np.nan)
sub['Charitable Contributions'] = sub['Charitable Contributions'].replace(0, np.nan)
sub['Amount Committed for Funding Social and Infrastructure Improvement Projects'] = sub['Amount Committed for Funding Social and Infrastructure Improvement Projects'].replace(0, np.nan)


In [9]:
geolocator = Nominatim(user_agent="nohagyousif@gmail.com")

def get_coordinates(row):
    """
    geocodes addresses using the Nominatim API

    parameters:
        row (series): a series containing address components,
                      including 'Address', 'City', 'State', and 'Zip Code'

    returns:
        Series: a series with two values - latitude and longitude
                if geocoding fails, returns NaN for both
    """

    address = {
        "street": row["Address"] if pd.notna(row["Address"]) else "",
        "city": row["City"] if pd.notna(row["City"]) else "",
        "state": row["State"] if pd.notna(row["State"]) else "",
        "postalcode": row["Zip Code"] if pd.notna(row["Zip Code"]) else "",
        "country": "US",
    }
    
    address = {k: v for k, v in address.items() if v}
    
    location = geolocator.geocode(address, timeout=10)
    if location:
        print(f"geocoded: {address} -> ({location.latitude}, {location.longitude})")
        return pd.Series([location.latitude, location.longitude])
    
    return pd.Series([np.nan, np.nan])

# geocode business addresses for mapping
sub[['Latitude', 'Longitude']] = sub.apply(get_coordinates, axis=1)

geocoded: {'street': '19 W 44TH STREET', 'city': 'NEW YORK', 'state': 'NY', 'postalcode': '10036', 'country': 'US'} -> (40.7553003, -73.9808892)
geocoded: {'street': '6320 CANOGA AVENUE', 'city': 'WOODLAND HILLS', 'state': 'CA', 'postalcode': '91367', 'country': 'US'} -> (34.18505236114312, -118.59727089790358)
geocoded: {'street': '245 SUMMER STREET', 'city': 'BOSTON', 'state': 'MA', 'postalcode': '02110', 'country': 'US'} -> (42.35126515, -71.05374054371987)
geocoded: {'street': '1818 MARKET STREET', 'city': 'PHILADELPHIA', 'state': 'PA', 'postalcode': '19103', 'country': 'US'} -> (39.95300985, -75.17100398724885)
geocoded: {'street': '299 SW CLAY ST.', 'city': 'PORTLAND', 'state': 'OR', 'postalcode': '97201', 'country': 'US'} -> (45.5129031, -122.6776166)
geocoded: {'street': '1430 BROADWAY', 'city': 'NEW YORK', 'state': 'NY', 'postalcode': '10018', 'country': 'US'} -> (40.7541258, -73.9866979)
geocoded: {'street': '1310 ROSENEATH ROAD', 'city': 'RICHMOND', 'state': 'VA', 'postalcod

In [11]:
sub.head()
# save modified 'sub' DataFrame to CSV
sub.to_csv('financial-statement-and-notes-2024-preprocessed.csv')